In [1]:
import numpy as np
import random
import subprocess
import time
from typing import List, Tuple

In [2]:
class Particle:
    def __init__(self, prompt_template: str, dim: int = 1):
        # Each particle's "position" is its current text prompt (string)
        self.position = np.array([prompt_template], dtype=object)
        #how much and in what way the prompt should change. Currently it`s defined as zero
        self.velocity = np.zeros(dim, dtype=float)  # Placeholder (not really used for text)
        # best version of the prompt so far
        self.best_position = self.position.copy()
        # assign a negative value at the initialization point inside the constructor.
        # hence, even a small imporvement becomes noticable
        self.best_score = -np.inf

In [3]:
def call_ollama(prompt: str, context_text: str, timeout: int = 60) -> str:
    """
    Calls Ollama using subprocess. Requires `ollama serve` running in background.
    """
    full_prompt = f"{prompt}\n\nContext:\n{context_text}"

    try:
        result = subprocess.run(
            ["ollama", "run", "phi3:mini", full_prompt],
                capture_output=True,
                text=True,
                encoding="utf-8",   # 👈 force UTF-8 decoding
                errors="replace",   # 👈 replace problematic characters instead of crashing
                timeout=timeout
)
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "[ERROR: Ollama call timed out]"
    except FileNotFoundError:
        return "[ERROR: Ollama binary not found in PATH]"

In [4]:
def fitness_function(prompt: str, context_text: str) -> Tuple[float, str]:
    """Evaluate how good the output is for a given prompt + context."""
    output = call_ollama(prompt, context_text)
    #Sometimes things go wrong (network hiccup, invalid prompt, etc.). If the AI’s reply starts
    # with [ERROR: ...], we say:"Okay, that’s a flop. Give it the lowest possible score (0.0),
    #and pass the error back.
    if output.startswith("[ERROR:"):  # error fallback
        return 0.0, output


    #Count how many words the output has. Why? Longer isn’t always better, but a one-word answer
    #like “Yes” is probably less useful.So this is basically: "More words = at least some effort.
    length_score = len(output.split())

    #Now check for important keywords inside the AI’s response. For example:
    #if it includes “FAQ,” “question,” “answer,” or “summary.” Each time one appears, we count it.
    # Why? Because those are the words we wanted the AI to emphasize in its answers.

    keyword_hits = sum([kw in output.lower() for kw in ["faq", "question", "answer", "summary"]])
    # custom score formulae.
    # Every word adds 0.1 points. (so a 50-word response = 5 points).
    # Every keyword hit adds 5 points. (so if it used "summary" and "answer" = +10 points).
    score = length_score * 0.1 + keyword_hits * 5
    return score, output

In [5]:
def pso_optimize(prompts: List[str], context_text: str, num_iterations: int = 3):
    particles = [Particle(prompt) for prompt in prompts]
    global_best_score = -np.inf
    global_best_prompt = None
    global_best_output = ""

    for it in range(num_iterations):
        print(f"\n=== Iteration {it+1}/{num_iterations} ===")

        for idx, p in enumerate(particles):
            # Evaluate
            score, output = fitness_function(p.position[0], context_text)
            print(f"  Particle {idx+1}: Score={score:.2f}")

            # Update personal best
            if score > p.best_score:
                p.best_score = score
                p.best_position = p.position.copy()

            # Update global best
            if score > global_best_score:
                global_best_score = score
                global_best_prompt = p.position[0]
                global_best_output = output

            # Mutate: tweak phrasing to explore search space
            if random.random() < 0.3:
                mutated = (
                    p.position[0]
                    .replace("Summarize", "Please summarize")
                    .replace("FAQ", "Frequently Asked Questions")
                    .replace("Q&A", "question and answer pairs")
                )
                p.position[0] = mutated

    return global_best_prompt, global_best_output, global_best_score


In [6]:
context_text = """
Large Language Models (LLMs) are revolutionizing natural language processing.
They can summarize documents, answer questions, and generate creative content.
However, results vary drastically depending on how prompts are phrased.
Finding the best formulation is known as prompt engineering.
"""

initial_prompts = [
    "Summarize the following document into an FAQ style with answers:",
    "Create a Frequently Asked Questions section from this text:",
    "Turn this document into 5 question-and-answer pairs, concise and clear:"
]


In [7]:
best_prompt, best_output, best_score = pso_optimize(
    initial_prompts,
    context_text,
    num_iterations=2
)


=== Iteration 1/2 ===
  Particle 1: Score=0.00
  Particle 2: Score=0.00
  Particle 3: Score=0.00

=== Iteration 2/2 ===
  Particle 1: Score=0.00
  Particle 2: Score=0.00
  Particle 3: Score=0.00


In [8]:
print("\n=== 🥇 Best Prompt Found ===")
print(best_prompt)

print("\n=== 📘 Sample Output (truncated) ===")
print(best_output[:600])

print("\nFitness Score:", best_score)



=== 🥇 Best Prompt Found ===
Summarize the following document into an FAQ style with answers:

=== 📘 Sample Output (truncated) ===
[ERROR: Ollama binary not found in PATH]

Fitness Score: 0.0
